In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, X, ids = load_csv_data(DATA_TRAIN_PATH)
tX = np.c_[np.ones(X.shape[0]),X]
print(len(y))
print(len(y[y > 0]))

250000
85667


In [3]:
import pandas as pd
from IPython.display import display

In [4]:
hb = pd.read_csv(DATA_TRAIN_PATH, sep=',')
pd.options.display.max_columns = None
hb = hb.drop(['Id', 'Prediction'], 1)
hb.describe()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,-49.023079,49.239819,81.181982,57.895962,-708.420675,-601.237051,-709.356603,2.373100,18.917332,158.432217,1.437609,-0.128305,-708.985189,38.707419,-0.010973,-0.008171,46.660207,-0.019507,0.043543,41.717235,-0.010119,209.797178,0.979176,-348.329567,-399.254314,-399.259788,-692.381204,-709.121609,-709.118631,73.064591
std,406.345647,35.344886,40.828691,63.655682,454.480565,657.972302,453.019877,0.782911,22.273494,115.706115,0.844743,1.193585,453.596721,22.412081,1.214079,1.816763,22.064922,1.264982,1.816611,32.894693,1.812223,126.499506,0.977426,532.962789,489.338286,489.333883,479.875496,453.384624,453.389017,98.015662
min,-999.000000,0.000000,6.329000,0.000000,-999.000000,-999.000000,-999.000000,0.208000,0.000000,46.104000,0.047000,-1.414000,-999.000000,20.000000,-2.499000,-3.142000,26.000000,-2.505000,-3.142000,0.109000,-3.142000,13.678000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
25%,78.100750,19.241000,59.388750,14.068750,-999.000000,-999.000000,-999.000000,1.810000,2.841000,77.550000,0.883000,-1.371000,-999.000000,24.591750,-0.925000,-1.575000,32.375000,-1.014000,-1.522000,21.398000,-1.575000,123.017500,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
50%,105.012000,46.524000,73.752000,38.467500,-999.000000,-999.000000,-999.000000,2.491500,12.315500,120.664500,1.280000,-0.356000,-999.000000,31.804000,-0.023000,-0.033000,40.516000,-0.045000,0.086000,34.802000,-0.024000,179.739000,1.000000,38.960000,-1.872000,-2.093000,-999.000000,-999.000000,-999.000000,40.512500
75%,130.606250,73.598000,92.259000,79.169000,0.490000,83.446000,-4.593000,2.961000,27.591000,200.478250,1.777000,1.225000,0.000000,45.017000,0.898000,1.565000,53.390000,0.959000,1.618000,51.895000,1.561000,263.379250,2.000000,75.349000,0.433000,0.503000,33.703000,-2.457000,-2.275000,109.933750
max,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,1852.462000,19.773000,1.414000,1.000000,764.408000,2.497000,3.142000,560.271000,2.503000,3.142000,2842.617000,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


In [5]:
hb = hb.replace(-999, np.nan)
hb.describe()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,211886.000000,250000.000000,250000.000000,250000.000000,72543.000000,72543.000000,72543.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,72543.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,150087.000000,150087.000000,150087.000000,72543.000000,72543.000000,72543.000000,250000.000000
mean,121.858528,49.239819,81.181982,57.895962,2.403735,371.783360,-0.821688,2.373100,18.917332,158.432217,1.437609,-0.128305,0.458290,38.707419,-0.010973,-0.008171,46.660207,-0.019507,0.043543,41.717235,-0.010119,209.797178,0.979176,84.822105,-0.003275,-0.012393,57.679474,-0.011845,-0.001582,73.064591
std,57.298157,35.344886,40.828691,63.655682,1.742226,397.699325,3.584362,0.782911,22.273494,115.706115,0.844743,1.193585,0.398681,22.412081,1.214079,1.816763,22.064922,1.264982,1.816611,32.894693,1.812223,126.499506,0.977426,60.662276,1.784546,1.813385,31.985782,2.031743,1.816950,98.015662
min,9.044000,0.000000,6.329000,0.000000,0.000000,13.602000,-18.066000,0.208000,0.000000,46.104000,0.047000,-1.414000,0.000000,20.000000,-2.499000,-3.142000,26.000000,-2.505000,-3.142000,0.109000,-3.142000,13.678000,0.000000,30.000000,-4.499000,-3.142000,30.000000,-4.500000,-3.142000,0.000000
25%,91.885250,19.241000,59.388750,14.068750,0.882500,111.977000,-2.629000,1.810000,2.841000,77.550000,0.883000,-1.371000,0.004000,24.591750,-0.925000,-1.575000,32.375000,-1.014000,-1.522000,21.398000,-1.575000,123.017500,0.000000,44.422500,-1.342000,-1.584000,37.312000,-1.612000,-1.576500,0.000000
50%,112.406000,46.524000,73.752000,38.467500,2.107000,225.885000,-0.244000,2.491500,12.315500,120.664500,1.280000,-0.356000,0.454000,31.804000,-0.023000,-0.033000,40.516000,-0.045000,0.086000,34.802000,-0.024000,179.739000,1.000000,65.561000,0.000000,-0.033000,47.902000,-0.010000,-0.002000,40.512500
75%,135.482000,73.598000,92.259000,79.169000,3.690000,478.226000,0.958000,2.961000,27.591000,200.478250,1.777000,1.225000,0.879000,45.017000,0.898000,1.565000,53.390000,0.959000,1.618000,51.895000,1.561000,263.379250,2.000000,103.342000,1.336000,1.562000,66.637000,1.589500,1.576000,109.933750
max,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,1852.462000,19.773000,1.414000,1.000000,764.408000,2.497000,3.142000,560.271000,2.503000,3.142000,2842.617000,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


In [6]:
hb = (hb - hb.mean()) / hb.std()
hb = hb.fillna(0)
hb = (hb - hb.mean()) / hb.std()
hb.describe()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05,2.500000e+05
mean,-1.060130e-17,1.880096e-17,1.267608e-17,-1.631406e-17,-8.583356e-18,-1.472245e-17,8.242296e-18,2.302158e-17,-8.014922e-18,-2.677325e-17,-6.988898e-17,-7.356959e-16,-5.556444e-17,4.973799e-17,-4.428102e-17,9.094947e-18,-2.725642e-17,6.201617e-17,2.148681e-17,1.023182e-17,-3.541345e-17,-3.922196e-17,1.925912e-15,7.162271e-18,-2.131628e-18,1.347189e-17,-4.868639e-17,1.516298e-17,-2.434319e-17,-5.082654e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.138668e+00,-1.393124e+00,-1.833343e+00,-9.095176e-01,-2.561276e+00,-1.671947e+00,-8.931170e+00,-2.765448e+00,-8.493204e-01,-9.708062e-01,-1.646192e+00,-1.077171e+00,-2.133972e+00,-8.347025e-01,-2.049313e+00,-1.724952e+00,-9.363372e-01,-1.964844e+00,-1.753563e+00,-1.264892e+00,-1.728199e+00,-1.550355e+00,-1.001790e+00,-1.166369e+00,-3.251405e+00,-2.227403e+00,-1.606480e+00,-4.100846e+00,-3.208624e+00,-7.454379e-01
25%,-4.965607e-01,-8.487457e-01,-5.337725e-01,-6.885043e-01,8.928376e-15,-5.058597e-15,-1.206744e-15,-7.192385e-01,-7.217697e-01,-6.990315e-01,-6.565422e-01,-1.041145e+00,4.104437e-15,-6.298241e-01,-7.528565e-01,-8.624289e-01,-6.474171e-01,-7.861712e-01,-8.617931e-01,-6.177055e-01,-8.635146e-01,-6.860080e-01,-1.001790e+00,-5.825900e-01,-3.107866e-01,-3.868959e-01,1.991853e-15,-2.136316e-17,2.495579e-17,-7.454379e-01
50%,-3.602904e-02,-7.683769e-02,-1.819794e-01,-3.052117e-01,8.928376e-15,-5.058597e-15,-1.206744e-15,1.512306e-01,-2.963986e-01,-3.264107e-01,-1.865768e-01,-1.907659e-01,4.104437e-15,-3.080222e-01,-9.906238e-03,-1.366657e-02,-2.784604e-01,-2.015248e-02,2.337156e-02,-2.102234e-01,-7.659549e-03,-2.376150e-01,2.130493e-02,4.465373e-15,-3.438908e-19,-7.296359e-17,1.991853e-15,-2.136316e-17,2.495579e-17,-3.321111e-01
75%,1.658339e-01,6.891572e-01,2.713048e-01,3.341892e-01,8.928376e-15,-5.058597e-15,-1.206744e-15,7.509156e-01,3.894166e-01,3.633864e-01,4.017679e-01,1.133815e+00,4.104437e-15,2.815259e-01,7.486937e-01,8.659198e-01,3.049996e-01,7.735346e-01,8.667000e-01,3.094045e-01,8.669570e-01,4.235753e-01,1.044400e+00,4.465373e-15,3.155231e-01,3.668152e-01,1.991853e-15,-2.136316e-17,2.495579e-17,3.761558e-01
max,2.028757e+01,1.813092e+01,3.106073e+01,4.362695e+01,6.499011e+00,2.148716e+01,9.069650e+00,4.228960e+00,1.264320e+02,1.464080e+01,2.170529e+01,1.292162e+00,2.522409e+00,3.237988e+01,2.065742e+00,1.733947e+00,2.327725e+01,1.994105e+00,1.705625e+00,8.514747e+01,1.739366e+00,1.418329e+01,2.067495e+00,2.203615e+01,3.256142e+00,2.244331e+00,3.852470e+01,4.122493e+00,3.211858e+00,1.591958e+01


In [7]:
tX = np.c_[np.ones(X.shape[0]), hb.to_numpy()]

## Do your thing crazy machine learning thing here :) ...

In [8]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using gradient descent.
    Uses MSE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    def compute_gradient(tx, n, e):
        return - tx.T @ e / n
    
    loss = 0
    w = initial_w
    n = y.shape[0]
    n2 = n*2
    
    for n_iter in range(max_iters):
        e = compute_e(y, tx, w)
        gradient = compute_gradient(tx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))

    return w, loss
'''
weights = np.array([])
for i in range(100):
    initial_w = np.full(tX.shape[1], i/100)
    max_iters = 100
    gamma = 0.3
    w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
    weights = np.append(weights, loss)
idx = np.argmin(weights)
'''
initial_w = np.zeros(tX.shape[1])
max_iters = 1000
gamma = 0.3
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
print(loss)

0.3404106662758695


In [9]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using gradient descent.
    Uses MAE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n, e):
        return 1/n * np.sum(np.abs(e))
    
    def compute_gradient(tx, n, e):
        e = y - tx @ w
    
        return -1/n*tx.T @ np.sign(e)
    
    loss = 0
    w = initial_w
    n = y.shape[0]
    n2 = n*2
    
    for n_iter in range(max_iters):
        e = compute_e(y, tx, w)
        gradient = compute_gradient(tx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))

    return w, loss

initial_w = np.zeros(tX.shape[1])
max_iters = 1000
gamma = 0.3
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
print(loss)

0.34672967168835184


In [10]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using stochastic gradient descent.
    Uses MAE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    def compute_gradient(tx, n, e):
        return - tx.T @ e / n
    
    loss = 0
    w = initial_w[:, np.newaxis]
    n = y.shape[0]
    n2 = n*2
    data_size = len(y)
    shuffled_indices = np.random.permutation(np.arange(data_size))
    shuffled_y = y[shuffled_indices]
    shuffled_tx = tx[shuffled_indices]
    shuffled_y = shuffled_y[:,np.newaxis]
    for n_iter, by, btx in zip(range(max_iters), shuffled_y, shuffled_tx):
        by = by[np.newaxis]
        btx = btx[np.newaxis, :]
        e = compute_e(by, btx, w)
        gradient = compute_gradient(btx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))
    return w, compute_loss(n2, compute_e(y, tx, w[:,0]))

initial_w = np.full(tX.shape[1], 0.1)
max_iters = 100000
gamma = 0.7
w, loss = least_squares_SGD(y, tX, initial_w, max_iters, gamma)
print(loss)

0.4915113296368542


In [11]:
def least_squares(y, tx):
    """
    Linear regression using normal equations.
    Use MSE loss function
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    w = la.solve(tx.T @ tx, tx.T @ y)
    
    return w, compute_loss(y.shape[0]*2, compute_e(y, tx, w))

w, loss = least_squares(y, tX)
print(loss)

0.34040945216155494


In [12]:
def ridge_regression(y, tx, lambda_):
    """
    Ridge regression using normal equations.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """ 
    
            
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    
    X = tx.T @ tx
    w = la.solve(X + lambda_ * (2*y.shape[0]) * np.eye(X.shape[0]), tx.T @ y)
    return w, compute_loss(y.shape[0]*2, compute_e(y, tx, w))



In [13]:
def ridge_regression_demo(tx, y):

    
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    
    rmse_tr = []
    for ind, lambda_ in enumerate(lambdas):
        weights, loss = ridge_regression(y, tx, lambda_)
        rmse_tr.append(np.sqrt(loss))

        print("lambda={l:.3f}, Training RMSE={tr:.3f}".format(l=lambda_, tr=rmse_tr[ind]))
        

ridge_regression_demo(tX, y)

w, loss = ridge_regression(y, tX, 0.001)
print(loss)

lambda=0.000, Training RMSE=0.583
lambda=0.000, Training RMSE=0.583
lambda=0.000, Training RMSE=0.583
lambda=0.000, Training RMSE=0.583
lambda=0.000, Training RMSE=0.583
lambda=0.001, Training RMSE=0.583
lambda=0.001, Training RMSE=0.583
lambda=0.003, Training RMSE=0.583
lambda=0.007, Training RMSE=0.584
lambda=0.016, Training RMSE=0.584
lambda=0.037, Training RMSE=0.586
lambda=0.085, Training RMSE=0.590
lambda=0.193, Training RMSE=0.598
lambda=0.439, Training RMSE=0.613
lambda=1.000, Training RMSE=0.634
0.3404162668067664


In [ ]:
def sigmoid(t):
    """apply sigmoid function on t."""
    return 1/(1 + np.exp(-t))
    
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    return np.sum(np.log(1 + np.exp(tx @ w)) - y * (tx @ w))


def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return tx.T @ (sigmoid(tx@w) - y)


def logistic_regression_step(y, tx, w):
    """return the loss, gradient"""
    return calculate_loss(y, tx, w), calculate_gradient(y, tx, w)
  

In [ ]:
# TODO
def logistic_regression(y, tx, lambda_):
    """
    Logistic regression using gradient descent or SGD.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """  
    def learning_by_gradient_descent(y, tx, w, gamma):
        """
        Do one step of gradient descen using logistic regression.
        Return the loss and the updated w.
        """
        loss = calculate_loss(y, tx, w)
        gradient = calculate_gradient(y,tx,w)
        w = w - gamma * gradient
        return loss, w
    
    # init parameters
    max_iter = 10000
    threshold = 1e-8
    #gamma = 0.01
    losses = []

    # build tx
    w = np.zeros((tx.shape[1], 1))
    y = y[:,np.newaxis]

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
     
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return w, losses[-1]
    

In [ ]:
lambda_ = 1e-8
w, loss = logistic_regression(y, tX, lambda_)

In [ ]:
# TODO
def reg_logistic_regression(y, tx, lambda_):
    """
    Regularized logistic regression using gradient descent or SGD.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """    
    def penalized_logistic_regression(y, tx, w, lambda_):
        """return the loss, gradient"""
        loss, gradient = logistic_regression_step(y, tx, w)
        loss     += 2 * lambda_ * la.norm(w)**2
        gradient += lambda_ * w

        return loss, gradient
    
    def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
        """
        Do one step of gradient descent, using the penalized logistic regression.
        Return the loss and updated w.
        """
        loss, gradient = penalized_logistic_regression(y, tx, w, lambda_) 
        w = w - gamma * gradient 

        return loss, w
    
    # init parameters
    max_iter = 10000
    gamma = 1e-8
    threshold = 1e-8
    losses = []

    w = np.zeros((tx.shape[1], 1))
    y = y[:,np.newaxis]

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
            
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    

In [ ]:
lambda_ = 0.000001
w, loss = reg_logistic_regression(y, tX, lambda_)

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
hbt = pd.read_csv(DATA_TEST_PATH, sep=',')
hbt = hbt.drop(['Id', 'Prediction'], 1)
hbt = hbt.replace(-999, np.nan)
hbt = (hbt - hbt.mean()) / hbt.std()
hbt = hbt.fillna(0)
hbt = (hbt - hbt.mean()) / hbt.std()
tX_test = hbt.to_numpy()

In [ ]:
OUTPUT_PATH = 'predictions.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w[1:], tX_test)#[:, [0, 1, 2, 3, 4, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]) # Selected desired columns
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
print(len(y_pred))
print(len(y_pred[y_pred > 0]))

# Submissions scores
Best score by technique

<ul>
    <li>MSE, gradient descent : 0.649</li>
    <li>MAE, gradient descent : 0.678 </li>
    <li>ridge regression      : 0.664</li>
</ul>
Best score after not being stupid with bias:

* MSE, GD: 
* MAE, GD: 0.639
* LSQ: 0.706
* R-REG: 0.730

Best score after normalizing test set + putting zero where unknown:

* LSQ: 0.723
* R-REG: 0.719

Feature expansion?